In [15]:
from flask import Flask, request, jsonify
import openai

In [16]:
# $5 will generate 15000 explanations

In [17]:
import os
try:
    with open("openai_token.txt", 'r') as file:
        token = file.readline().strip()
except FileNotFoundError:
    print("File not found:", file_path)

In [18]:
prompt_question_1 = "in a paragraph, explain the rationale behind the last move, where all previous moves are:"
prompt_question_2 = ", and last move is "

In [9]:
from openai import OpenAI
client = OpenAI(api_key = token)
setup  = prompt_question_1 + df.iloc[1_000]["prev_moves"] + prompt_question_2 + df.iloc[1_000]["last_move_fill"]
print(setup)
# temp = "e4 e5 Nf3 Nc6 Bc4 h6 O-O d6 d3 a6 a4 Bg4 Be3 Qf6 h3 Bxf3 gxf3 Be7 f4 exf4 Bd2 f3 Bc3"
stream = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
#     model="gpt-4",
    messages=[{"role": "user", "content": setup}],
    stream=True,
)
print()
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

in a paragraph, explain the rationale behind the last move, where all previous moves are:d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd3 Bxd3 Qxd3 O-O Bg5 Ng4, and last move is Bxe7

The last move, Bxe7, captures the opponent's bishop on e7 with the bishop. This move aims to eliminate a strong piece from the opponent's side while forcing them to recapture with their queen, disrupting their pawn structure. By doing this, the player can gain a material advantage and potentially create a weakness in the opponent's position that can be exploited in further moves. Additionally, by exchanging pieces, the player reduces the opponent's attacking chances and increases their own chances of a successful endgame. Overall, the move Bxe7 is a strategic decision to improve the player's position and increase their chances of winning the game.

In [19]:
import pandas as pd

In [20]:
df = pd.read_csv("prev_last_5000.csv")

In [21]:
# df = pd.DataFrame(df["progressive_moves"])
# df = pd.DataFrame(df["prev_moves"], df["last_move_fill"])
df.head()

,Unnamed: 0,prev_moves,last_move_fill
0,0,d4,d4
1,1,d4,d5
2,2,d4 d5,c4
3,3,d4 d5 c4,c6
4,4,d4 d5 c4 c6,cxd5


In [22]:
# df.iloc[12_000]["progressive_moves"]

In [23]:
df.shape

(5000, 3)

In [24]:
df["explanation"] = ""
df.shape

(5000, 4)

In [25]:
df.tail()

,Unnamed: 0,prev_moves,last_move_fill,explanation
4995,4995,d4 Nf6 Nf3 e6 e3 b6 Bd3 Bb7 c4 c5 Qc2,Bxf3,
4996,4996,d4 Nf6 Nf3 e6 e3 b6 Bd3 Bb7 c4 c5 Qc2 Bxf3,gxf3,
4997,4997,d4 Nf6 Nf3 e6 e3 b6 Bd3 Bb7 c4 c5 Qc2 Bxf3 gxf3,cxd4,
4998,4998,d4 Nf6 Nf3 e6 e3 b6 Bd3 Bb7 c4 c5 Qc2 Bxf3 gxf...,exd4,
4999,4999,d4 Nf6 Nf3 e6 e3 b6 Bd3 Bb7 c4 c5 Qc2 Bxf3 gxf...,Nc6,


In [26]:
# df_prompt.at[0, "explanation"] = "checking"
# df_prompt

In [29]:
import time

for i in range(5000):
    try:
        stream = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=[{"role": "user", 
                       "content": prompt_question_1 + df.iloc[i]["prev_moves"] + prompt_question_2 + df.iloc[i]["last_move_fill"]
                      }],
            stream=True
        )
        
        explanation = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                explanation += chunk.choices[0].delta.content
                
        df.at[i, "explanation"] = explanation
        
        # every 100 will output
        if (i + 1) % 100 == 0:
            print(f"{i}. {df.iloc[i]['prev_moves']}--{df.iloc[i]['last_move_fill']}, explanation: {explanation}")
    
    except Exception as e:
        print(f"Error occurred while processing index {i}: {str(e)}")
        print("Retrying after 5 seconds...")
        time.sleep(5)  # Wait for 5 seconds before retrying
        continue


99. d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7--O-O, explanation: The last move, O-O (castling kingside), was a logical choice because it aims to improve the safety of White's king by moving it to a more secure position. Castling also connects White's rooks, allowing for better coordination and potential attacking opportunities. Additionally, by castling, White effectively moves the king away from the center, which may be beneficial as it reduces the risk of potential threats from Black's pieces. Overall, castling is a key maneuver in chess that helps to fortify the king's position and prepare for future strategic plans.
199. e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 Nf6 Bg5 O-O b5 Nc5 Bxf6 Bxf6 Bd3 Qd7 O-O Nxd3 Qxd3 c6 a4 cxd5 Nxd5 Qe6 Nc7 Qg4 Nxa8 Bd7 Nc7 Rc8 Nd5 Qg6 Nxf6+ Qxf6 Rfd1 Re8 Qxd6 Bg4 Qxf6 gxf6 Rd3 Bxf3 Rxf3 Rd8 Rxf6 Kg7 Rf3--Rd2, explanation: The move Rd2 was played to double up the rooks on the d-file, putting pressure on the f2 pawn and creating potential threats a

1299. e4 c5 d4 cxd4 Qxd4 Nc6 Qa4 Nf6 Nc3 g6 Nf3 Bg7 Bb5 O-O O-O a6 Bg5 axb5 Qxa8 d5 Rab1 dxe4 Rbd1 Qa5 Qxa5 Nxa5 Nxb5 exf3 gxf3 Nc4 Na7 Nxb2 Rb1 Nc4 Rb4 Nd2 Rd1 Nxf3+ Kf1 Nxh2+ Kg1 Nf3+ Kg2 Nxg5 Rbd4 Bh3+ Kg3 Nge4+ Kxh3 Nxf2+ Kg2 Nxd1 Rxd1 Ne4--Rd4, explanation: The last move, Rd4, underscores the player's aggressive strategy by targeting the knight on d2 and potentially threatening to capture it on the next move. By placing the rook on d4, the player is not only securing control over an open file but also potentially setting up a tactic to win material. This move demonstrates an awareness of positional advantages and the importance of maintaining pressure on the opponent's pieces. Additionally, by attacking the knight, the player may be seeking to disrupt their opponent's coordination or force them to make defensive moves, further consolidating their strategic position on the board.
1399. e4 c5 Nf3 d5 exd5 Qxd5 Nc3 Qe6+ Qe2 Qxe2+ Bxe2 Nf6 O-O g6 d3 Bg7 Nb5 Na6 Bf4 O-O d4 Bd7 dxc5 Bxb5

2499. d4 d5 Bf4 Nc6 Nf3 f6 e3 g5 Bg3 a6 Bd3 g4--Nfd2, explanation: The last move, Nfd2, is a strategic one as it relocates the knight from its current position to a potentially stronger one. By moving the knight to d2, White's pieces are better centralized and the knight has the flexibility to both support its own pawn structure and potentially jump into the game in the future. This move also maintains control over key central squares and prepares for possible future attacks or defenses. Overall, this move strengthens White's position and sets the stage for further development and tactical opportunities.
2599. d4 Nf6 e3 g6 f4 Bg7 Nf3 d5 c4--O-O, explanation: The last move, O-O, was played to complete the king's side development by castling. By moving the king to safety on the kingside, White not only protects the king but also connects the rooks and prepares for the middle game. This move also opens up the g1 square for the rook to potentially be activated on the g-file. Furthermore, c

3799. d4 d5 Bf4 Nf6 Nf3 c5 e3 Bg4 Nbd2 cxd4 Bb5+ Nc6 exd4 Qb6 Bxc6+ bxc6 Nb3 e6 c3 Be7 h3 Bh5 g4--Bg6, explanation: The last move, Bg6, is likely made in order to break the pin on the knight on f3 created by the bishop on g4. By moving the bishop to g6, the tension between the bishop and knight is relieved and the bishop is now free to move without worrying about being taken. Additionally, the bishop on g6 now has the potential to be placed on a more active square in the future, potentially influencing the center of the board or supporting other pieces in future attacks. Overall, the move Bg6 helps to improve the position of the pieces and maintain control over the board.
3899. d4 e6 Bf4 d5--Nf3, explanation: The last move of Nf3 in this sequence is a strategic decision to further develop the knight and reinforce the central control established by the earlier moves. By moving the knight to f3, white is preparing to castle kingside and potentially exert pressure on the center of the boa

4999. d4 Nf6 Nf3 e6 e3 b6 Bd3 Bb7 c4 c5 Qc2 Bxf3 gxf3 cxd4 exd4--Nc6, explanation: The last move, Nc6, is aiming to centralize the knight and strengthen Black's position. By placing the knight on c6, Black is attacking White's e5 pawn, putting pressure on White's central pawn structure. Additionally, the knight on c6 could potentially be repositioned to d4 or b4 in the future to target key squares on the board. Overall, this move is strategic in nature as it aims to develop and improve the position of Black's pieces while exerting pressure on White's position.


In [ ]:
# df_prompt.iloc[50]

In [ ]:
# df_prompt.to_csv("few_exp.csv")

In [30]:
df.to_csv("div_move_exp_5000.csv")